In [ ]:
import requests
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import git
from git import Repo
from PIL import Image
import io
import subprocess

last_update_id = None
TOKEN = ''  #creds
chat_id = ''  #creds


def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
    response = requests.get(url)
    print(f"Sent message: {message}, Response: {response.json()}")

def send_telegram_image(image_path):
    url = f"https://api.telegram.org/bot{TOKEN}/sendPhoto"
    with open(image_path, 'rb') as image_file:
        files = {'photo': image_file}
        data = {'chat_id': chat_id}
        response = requests.post(url, files=files, data=data)
    print(f"Sent image: {image_path}, Response: {response.json()}")

def get_updates(offset=None):
    url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    params = {'offset': offset, 'timeout': 30}
    response = requests.get(url, params=params).json()
    return response.get('result', [])

def clear_previous_messages():
    global last_update_id
    updates = get_updates()
    if updates:
        last_update_id = updates[-1]['update_id'] + 1
    print(f"Cleared previous messages. Last update ID: {last_update_id}")

def get_cuda_options():
    p = subprocess.run(['nvidia-smi','-q'], capture_output=True, text=True)
    pret=p.returncode
    if pret!=0:
        print("Device doesnt have GPU or CUDA drivers error")
        send_telegram_message("Device doesnt have GPU or CUDA drivers error")
        return
    pop=p.stdout
    for popline in pop.splitlines():
      send_telegram_message(popline)


def preprocess_image(imgbytes):
    image_stream=io.BytesIO(imgbytes)
    image=Image.open(image_stream).convert('L')
    transform = transforms.Compose([
        transforms.Resize((28, 28)),  # Resize to the input size expected by the model
        transforms.ToTensor(),  # Convert image to tensor
        transforms.Normalize((0.5,),(0.5,))  # Normalize with ImageNet mean and std
    ])
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

def test_model(fuid):
    global model
    url=f"http://api.telegram.org/bot{TOKEN}/getFile?file_id={fuid}"
    response=requests.get(url)
    fpath=response.json()['result']['file_path']
    url=f"http://api.telegram.org/file/bot{TOKEN}/{fpath}"
    response=requests.get(url)
    image_tensor=preprocess_image(response.content)
    model.eval()
    res=model(image_tensor)
    print(res)
    predicted_class = res.argmax(dim=1).item()
    print(f'Predicted class: {predicted_class}')
    send_telegram_message(f'Predicted class: {predicted_class}')


def request_user_input(prompt):
    global last_update_id
    send_telegram_message(prompt)
    start_time = time.time()
    timeout = 300
    while time.time() - start_time < timeout:
        updates = get_updates(last_update_id)
        for update in updates:
            last_update_id = update['update_id'] + 1
            if 'message' in update and 'text' in update['message']:return update['message']['text'].lower()
            if 'message' in update and 'photo' in update['message']:test_model(update['message']['photo'][0]['file_id'])
        time.sleep(1)
    send_telegram_message("No input received within 5 minutes. Using default value.")
    return None

def get_hyperparameters():
    choice = request_user_input("Do you want to input hyperparameters? (yes/no)")
    if choice == 'yes':
        learning_rate =float(request_user_input("Enter learning rate (e.g., 0.001):") or 0.001)
        batch_size = int(request_user_input("Enter batch size (e.g., 64):") or 64)
        num_epochs = int(request_user_input("Enter number of epochs (e.g., 2):") or 2)
        hidden_size = int(request_user_input("Enter hidden layer size (e.g., 512):") or 512)
    else:
        learning_rate = 0.001
        batch_size = 64
        num_epochs = 2
        hidden_size = 512
    send_telegram_message(f"Using hyperparameters: learning_rate={learning_rate}, batch_size={batch_size}, num_epochs={num_epochs}, hidden_size={hidden_size}")
    return learning_rate, batch_size, num_epochs, hidden_size

class SimpleNet(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, hidden_size),nn.ReLU(),nn.Linear(hidden_size, 10))
    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

def train_model(num_epochs,train_loader,model,criterion,optimizer):
    model.train()
    epoch_losses=[]
    for epoch in range(num_epochs):
        epoch_loss=0
        for _, (data,targets) in enumerate(train_loader):
            outputs=model(data)
            loss=criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss+=loss.item()
        epoch_loss/=len(train_loader)
        epoch_losses.append(epoch_loss)
        print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')
        send_telegram_message(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')
    return epoch_losses

def check_file_sizes():
    max_file_size = 100*1024*1024
    acceptable_files = []
    for root, dirs, files in os.walk('.'):
        for file in files:
            file_path = os.path.join(root, file)
            if os.path.getsize(file_path) <= max_file_size:acceptable_files.append(file_path)
    return acceptable_files

def push_to_github(repo_url, is_private):
    try:
        if os.path.exists('.git'):repo = Repo('.')
        else:
            repo = Repo.init('.')
            repo.create_remote('origin', url=repo_url)
        repo.git.add(A=True)
        repo.index.commit("Update from Telegram bot")
        origin = repo.remote('origin')
        origin.push()
        send_telegram_message("Successfully pushed to GitHub repository.")
    except Exception as e:send_telegram_message(f"Error pushing to GitHub: {str(e)}")

# Clear previous messages
clear_previous_messages()

def proginit():
    global model
    learning_rate, batch_size, num_epochs, hidden_size = get_hyperparameters()
    transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
    train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    model = SimpleNet(hidden_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    send_telegram_message("Model training has started.")

    epoch_losses = train_model(num_epochs, train_loader, model, criterion, optimizer)
    final_loss = epoch_losses[-1]

    send_telegram_message(f"Training complete. Final loss: {final_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(epoch_losses, marker='o', linestyle='-', color='b')
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.savefig('training_loss_plot.png')
    plt.close()

    send_telegram_image('training_loss_plot.png')

while True:
    proginit()
    # GitHub integration
    use_github = request_user_input("Do you want to use GitHub? (yes/no)")
    if use_github == 'yes':
        acceptable_files = check_file_sizes()
        send_telegram_message(f"Files acceptable for GitHub push: {', '.join(acceptable_files)}")
        push_decision = request_user_input("Do you want to push these files to GitHub? (yes/no)")
        if push_decision == 'yes':
            repo_url = request_user_input("Enter your GitHub repository URL:")
            is_private = request_user_input("Is this a private repository? (yes/no)")=='yes'
            push_to_github(repo_url, is_private)

    command = request_user_input("Enter 'rerun' to train again with new parameters, or 'stop' to end the program, 'cuda' to get CUDA status and stop, or an image to test model:")
    if command == "stop":
        send_telegram_message("Training stopped by user command.")
        break
    elif command == "rerun":
        send_telegram_message("Rerunning the training with new parameters.")
        continue
    elif command=="cuda":
        get_cuda_options()
        break
    else:
        send_telegram_message("Invalid command. Stopping the program.")
        break
